## 作业二

### 数据预处理

In [7]:
import pandas as pd
import numpy as np

file_path = 'data/netflix_train.txt'
data_train = pd.read_csv(file_path, sep=r'\s+', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'])
print(data_train.head())

users_train = data_train['user_id'].unique()
movies_train = data_train['movie_id'].unique()

X_train = pd.DataFrame(0, index=users_train, columns=movies_train)

for row in data_train.itertuples():
    X_train.at[row.user_id, row.movie_id] = row.rating

print("Matrix for Train:")
print(X_train)

# X.to_csv('user_movie_matrix.csv')
# missing_mask.to_csv('missing_mask.csv')


   user_id  movie_id  rating   timestamp
0   305344         1       1  2004-02-08
1   305344         3       2  2003-03-23
2   305344         4       1  2003-12-13
3   305344         5       1  2004-12-15
4   305344         6       1  2003-09-23
Matrix for Train:
         1     3     4     5     6     7     8     10    15    17    ...  \
305344      1     2     1     1     1     2     1     1     5     2  ...   
387418      1     2     0     1     2     1     0     0     0     3  ...   
2439493     0     0     1     0     1     0     1     1     0     1  ...   
1664010     5     0     3     5     4     3     4     4     4     0  ...   
2118461     5     4     3     5     4     3     0     3     0     3  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
1688557     0     0     0     0     0     0     0     0     0     0  ...   
959444      0     0     0     0     0     0     0     0     0     0  ...   
1581163     0     0     0     0     0     0     3   

In [10]:
file_path = 'data/netflix_test.txt'
data_test = pd.read_csv(file_path, sep=r'\s+', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'])
print(data_test.head())

users_test = data_test['user_id'].unique()
movies_test = data_test['movie_id'].unique()
print(len(data_test['movie_id'].unique()))
X_test = pd.DataFrame(0, index=users_test, columns=movies_test)
print(X_test.shape)
# for row in data_test.itertuples():
#     X_test.at[row.user_id, row.movie_id] = row.rating

# print("Matrix for Test:")
# print(X_test)

   user_id  movie_id  rating   timestamp
0  2123534      6134       4  2005-05-31
1  2123534       677       1  2005-06-06
2  2123534      3182       4  2005-06-06
3  2123534      7882       4  2005-06-27
4  2123534      6099       1  2005-05-31
9983
(10000, 9983)
